<a href="https://colab.research.google.com/github/mzaman202006/DA_project/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install beautifulsoup4

In [ ]:
pip install requests bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

from openpyxl import load_workbook

In [ ]:
import pandas
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


# Collect Movie Data


In [ ]:
url= 'https://www.imdb.com/title/tt8380776/'
r = requests.get(url)
movies_soup = BeautifulSoup(r.content, 'html.parser')
title = movies_soup.find('div', attrs={'class': 'title_wrapper'})
year =''
final_name =''
genre_name_str =''
#print(title)
release_date_final = ''
final_rating = ''
final_plot= ''

if(title is not None):
  year = (title.find('h1').find('span').text) [1:-1]
  
  #print(year)
  title_name = (title.find('h1').text)

  final_name = title_name[:len(title_name)-7]

  #print(final_name)

  genre_name = [tag.text for tag in title.find_all('a', attrs={'class': ''})if tag.attrs['href'].startswith('/search/title?genres') ] 
  #print(genre_name)
  genre_name_str = ','.join(genre_name)

  release_date = [tag.text for tag in title.find_all('a', attrs={'class': ''})if tag.attrs['href'].endswith('releaseinfo') ] 
  release_date_final = (release_date[0].strip())

rating = movies_soup.find('div', attrs={'class': 'ratingValue'})
#print(rating.find('strong').attrs['title'])
if(rating is not None):
 final_rating  = rating.find('span', attrs={'itemprop': 'ratingValue'}).text
 #print(final_rating)

plot  = movies_soup.find('div', attrs={'class': 'plot_summary_wrapper'})
if(plot is not None):
 final_plot_0 = plot.find('div', attrs={'class': 'summary_text'})
 final_plot = final_plot_0.text.strip()

#Review Data

In [ ]:


#base_url = "https://www.imdb.com"
review_links_ = url+ 'reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0'
#print("There are a total of " + str(len(movie_links)) + " movie user reviews")
#print("Displaying 10 user reviews links")
#movie_links_

# get a list of soup objects
index =0
rows = []

   
soup = BeautifulSoup(requests.get(review_links_).content, 'html.parser')
total_review = soup.find('div', attrs={'class': 'header'}).find('span').text
total_review_count = re.findall('[0-9]+', total_review.replace(",", ""))
Review_tags_ = soup.findAll('div', attrs={'class': 'review-container'})
#print(first_movie_tags_)
if(Review_tags_ is not None):
  for movie_review_holder in Review_tags_:
    review = movie_review_holder.find('div', attrs={'class': 'text show-more__control'})
    #print(review.text)
    title = movie_review_holder.find('a', attrs={'class': 'title'})
    #print(title.text)
    #print(title.attrs['href'])

    r_date = movie_review_holder.find('span', attrs={'class': 'review-date'})
    #print(r_date.text)

    user_l = movie_review_holder.find('span', attrs={'class': 'display-name-link'})
    user_a = user_l.findChildren("a" , recursive=False)
    #print(user_a[0])
    rating_final =0
    rating =  movie_review_holder.find('span', attrs={'class': 'rating-other-user-rating'})
    if (rating is not None):
      rr_f =rating.findAll('span')
      rating_final =  rr_f[0].text
    #print(rating_final)
    
    help = movie_review_holder.find('div', attrs={'class': 'actions'}).text
    #print(help)
    numbers = re.findall('[0-9]+', help.strip().replace(",", ""))
    #print(total_review_count[0])
    numbers_helpful_str = '/'.join(numbers)
    #print(numbers_helpful_str)
    #print(','.join(total_review_count))
    rows.append([review_links_,year,final_name,genre_name_str,release_date_final,final_rating,final_plot,
    total_review_count[0],title.text,title.attrs['href'],review.text ,r_date.text,user_a[0],rating_final,numbers_helpful_str])

df = pd. DataFrame(rows, columns=["URL", "YEAR","MOVIE","Genre","Release date","rating","plot",
                                    "total review","Title","LINK","REVIEW","date","Author","RATING","helpful"])

#print(df['REVIEW'])
#print(df['Title'])
#print(df['date'])
#print(df['RATING'])

In [11]:
#print(rows)
writer = pd.ExcelWriter('/content/sample_data/data.xlsx', engine='openpyxl')
# try to open an existing workbook
writer.book = load_workbook('/content/sample_data/data.xlsx')
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
# read existing file
reader = pd.read_excel(r'/content/sample_data/data.xlsx')
# write out the new sheet
df.to_excel(writer,index=False,header=False,startrow=len(reader)+1)

writer.close()